In [43]:
using Random
using Distributions
using FLoops
using Base.Threads
using BenchmarkTools

include("data.jl")
include("util.jl")

include("intervals/permutation.jl")
include("intervals/bootstrap.jl")
include("intervals/t.jl")

Threads.nthreads()

8

In [44]:
dtype = Float32
seed = 123

# DATA CONFIG

alpha = 0.05

# data
B  = 100   # num. coverage probabilities per boxplot
S  = 4300  # num. samples per coverage probability
nx = 100   # size of group 1
ny = 50    # size of group 2
nperm = try
    binomial(nx+ny, nx)
catch OverflowError
    Inf
end
    
if nperm > 30_000
    Random.seed!(123)
    px, py = partition(nx, ny, 10_000)
else
    px, py = partition(nx, ny)
end

bits = vcat(ones(Int, nx), zeros(Int, ny))
addx, addy = bits[px], bits[py]
px[:, 1:5]

100×5 Matrix{Int64}:
   1   79   71   14   60
   2   89   99  143   84
   3  134   98   97   34
   4   32  106  142   31
   5   81   65  109  100
   6   62   83   27   94
   7   13  115  144  135
   8  141  121   56   82
   9   91  145   98   22
  10   56  120  138   87
  11   84   30   90   58
  12  128   97  150   66
  13   18   31  129  130
   ⋮                 
  89  127    3   65   83
  90  110  102  126   75
  91  133   19   23   28
  92  144    8   44   99
  93   87  114  133   37
  94   98   41   60    8
  95  103  107  105   38
  96  105  143  131   11
  97   75   43   20   59
  98   82   16   21  140
  99   65  119   37    7
 100   46    9   55   50

In [45]:
# POPULATION SETTINGS

Random.seed!(123)

distrTypeX = Gamma{dtype}
X_shape = random(Uniform(2, 4), B)
X_scale = random(Uniform(0.5, 2), B)
distrX = map(distrTypeX, X_shape, X_scale)

distrTypeY = Gumbel{dtype}
Y_loc = random(Uniform(0, 1), B)
Y_scale = random(Uniform(2, 4), B)
distrY = map(distrTypeY, Y_loc, Y_scale)
;

In [46]:
deltas = @. mean(distrX) - mean(distrY)

@show distrX[1:2]
@show distrY[1:2]
@show deltas[1:2];

distrX[1:2] = Gamma{Float32}[Gamma{Float32}(α=3.813f0, θ=1.931f0), Gamma{Float32}(α=2.887f0, θ=1.769f0)]
distrY[1:2] = Gumbel{Float32}[Gumbel{Float32}(μ=0.965f0, θ=2.056f0), Gumbel{Float32}(μ=0.071f0, θ=2.495f0)]
deltas[1:2] = Float32[5.211148, 3.5959504]


In [47]:
Random.seed!(123)
xs = [dtype.(rand(distrX[i], nx, S)) for i in 1:B]
ys = [dtype.(rand(distrY[i], ny, S)) for i in 1:B]
@show size(ys)
ys[2]

size(ys) = (100,)


50×4300 Matrix{Float32}:
  4.96632    2.68948     3.39481    …  -0.654883    0.649917   -0.547117
  4.1306     0.51833    -0.340174       3.00418    -1.37342     3.25588
 -3.05929    5.30728    -0.341889      -3.47024    -0.378375    2.54732
  5.46767    2.42243    -0.265613      -0.574028    2.42536    -1.53447
  7.25734    9.90195     2.97764       -2.57248     2.37158    -1.15894
  3.56301    3.03044     2.02682    …   3.09028     0.404494   -0.499147
  4.00801    3.27124    -2.31258       -0.312584   -0.0365355  -2.49059
  1.70405    4.81771     0.102927       1.85326     0.29744    -0.631769
 -1.32989    6.57513    -0.201606       2.74437    -0.937224    0.759019
  2.11959    9.82986     1.35133        0.0537991  -1.30835     4.69618
  6.4245     4.28912     0.0536875  …   5.30297    -2.42861     6.44185
  2.01805    2.83512    -1.29911       27.0076      4.24639    -1.04852
  3.12121    1.61074     0.75042       11.8885      0.0403012   1.72527
  ⋮                                

In [48]:
# flatten into 3D matrix
X = reshape(hcat(xs...), nx, S, B)
Y = reshape(hcat(ys...), ny, S, B)
Y[:,:,2]

50×4300 Matrix{Float32}:
  4.96632    2.68948     3.39481    …  -0.654883    0.649917   -0.547117
  4.1306     0.51833    -0.340174       3.00418    -1.37342     3.25588
 -3.05929    5.30728    -0.341889      -3.47024    -0.378375    2.54732
  5.46767    2.42243    -0.265613      -0.574028    2.42536    -1.53447
  7.25734    9.90195     2.97764       -2.57248     2.37158    -1.15894
  3.56301    3.03044     2.02682    …   3.09028     0.404494   -0.499147
  4.00801    3.27124    -2.31258       -0.312584   -0.0365355  -2.49059
  1.70405    4.81771     0.102927       1.85326     0.29744    -0.631769
 -1.32989    6.57513    -0.201606       2.74437    -0.937224    0.759019
  2.11959    9.82986     1.35133        0.0537991  -1.30835     4.69618
  6.4245     4.28912     0.0536875  …   5.30297    -2.42861     6.44185
  2.01805    2.83512    -1.29911       27.0076      4.24639    -1.04852
  3.12121    1.61074     0.75042       11.8885      0.0403012   1.72527
  ⋮                                

In [56]:
function save_ci_results(results, methodId, B, S, pooled=nothing, two_sided=nothing; prefix="", dir="./")
    averages = []

    for batchId in 1:B
        batch = results[methodId, batchId, :]
        coverage = sum([j for (j, _) in batch]) / S
        width = sum([j for (_, j) in batch]) / S
        push!(averages, (coverage, width))
    end
    if isnothing(two_sided)
        save(averages, distrX[1:B], distrY[1:B], alpha, prefix=prefix, dir=dir)
    else
        alpha_ = two_sided ? alpha : alpha / 2
        save(averages, distrX[1:B], distrY[1:B], alpha_, pooled, two_sided, prefix=prefix, dir=dir)
    end
end

function save_permutation_results(results, B, S; prefix="", dir="./")
    i = 1
    per_method = []
    for two_sided in [true, false]
        for pooled in [true, false]
            if i in [1, 3]  # use only pooled for now
                save_ci_results(results, i, B, S, pooled, two_sided, prefix=prefix, dir=dir)
            end
            i += 1
        end
    end
end

save_permutation_results (generic function with 1 method)

In [57]:
function cache(groups, masks)
    mean_og = mean(groups, dims=1)
    var_og = var(groups, dims=1)
    nshift = sum(masks, dims=1)
    shift_sum = sum(groups .* masks, dims=1)
    return P(mean_og, var_og, nshift, shift_sum, size(groups, 1))
end

cache (generic function with 1 method)

In [59]:
x = X[:,1,1]
y = Y[:,1,1]
pooled = vcat(x, y)
xs = pooled[px]
ys = pooled[py]
xcache = cache(xs, addx)
ycache = cache(ys, addy)

wide, narrow = t_estimates(x, y, true)
@time permInterval(xcache, ycache, wide, narrow, true, alpha, twoSided, twoSided, 0.005)

  0.016630 seconds (2.29 k allocations: 14.835 MiB)


(4.694457650790023, 7.0463708633090985)

In [ ]:
T = Threads.nthreads()
results = Array{Union{Tuple, Nothing}, 3}(nothing, 6, B, S)

#@time Threads.@threads for (i,j) in collect(Iterators.product(1:B, 1:S)) # 15.52 sec on (B,S) = (5, 1800)
@time @floop ThreadedEx(basesize=div(B*S, T)) for b in 1:1, s in 1:1000
    @inbounds x = X[:,s,b]
    @inbounds y = Y[:,s,b]
    
    pooled = vcat(x, y)
    xs = @inbounds pooled[px]
    ys = @inbounds pooled[py]
    xcache = cache(xs, addx)
    ycache = cache(ys, addy)
    wide, narrow = t_estimates(x, y, true)
 
    results[1, b, s] = permInterval(xcache, ycache, wide, narrow, deltas[b], true, alpha, twoSided, twoSided, 0.0005)
    #results[2, b, s] = permInterval(xcache, ycache, wide, narrow, deltas[b], false, alpha, twoSided, twoSided, 0.0005)
    results[3, b, s] = permInterval(xcache, ycache, wide, narrow, deltas[b], true, alpha/2, greater, smaller, 0.0005)
    #results[4, b, s] = permInterval(xcache, ycache, wide, narrow, deltas[b], false, alpha/2, greater, smaller, 0.0005)
    
    #"""
    #results[5, b, s] = bootstrap(x, y, deltas[b], alpha, nsamples=10_000)
    #results[6, b, s] = tconf(x, y, deltas[b], alpha, false)
    #"""
end

dir = "../results/" * string(nx) * "_" * string(ny) * "/3/"
# save_permutation_results(results, B, S; dir=dir)
#save_ci_results(results, 5, B, S; prefix="bs", dir=dir)
#save_ci_results(results, 6, B, S; prefix="t", dir=dir)

In [ ]:
b = 10
s = 100
[sum(x for (x, _) in results[4, b, 1:s]) / s for b in 1:b]

10-element Vector{Float64}:
 0.95
 0.98
 0.95
 0.94
 0.95
 0.93
 0.95
 0.92
 0.94
 0.97